# Pre-processing Tutorial

## Introduction
In this tutorial, you'll learn how to use Pandas to perform some basic pre-processing tasks that you may need to carry out to clean your data for the project before you can import it to your database. 

Specifically, the tutorial covers:


*   Removal of rows with missing values
*   Replacement of missing values with default values
*   Dectection of entity resolution problems
*   Simple entity resolution
*   Removal of unpaired entities
*   Replacement of categorical variables with indicators
*   Identification of candidate indices
*   Data exportation

To edit and run the code throughout the tutorial, open the notebook in "playground mode" using the button in the upper right corner.

After going through the tutorial, try the accompanying exercises to practice what you learned. 

## Setup

Run the cell below to import necessary modules. 

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive

The tutorial uses two datasets:
1. The [subset of U.S. census data](https://drive.google.com/open?id=1WB60Q6VJYyjbE8h8WED2mfi8Ae8LzoRU) maintained by the UCI Machine Learning Repository, which we used in the last tutorial.
2. A [modified life expectancy dataset](https://drive.google.com/open?id=1DrjuDzQ_rcgULkm3zQqwLQQGIuDInoj5) compiled by the World Health Organization.

For the purpose of the tutorial, imagine we're planning to build an application that will draw on both these datasets. It could, for example, help users learn about the relationship between how immigrants fair in the United States and health indicators in their native country. 

To prepare the datasets for ingestion into the database, we need to:
1. Clean missing / misentered values
2. Detect and solve entity resolution problems
3. Replace categorical variables with numeric indicators for efficiency
4. Export the data

Before doing any of this, we need to import the datasets to _pandas_ `DataFrames`. We'll follow the data importation procedure outlined in the [EDA tutorial](https://drive.google.com/open?id=1Cy3izai9zLQYTCTQF9IwkcuLmNArcKZO). Add the datasets to your Google Drive using the links above, then mount your Drive to the notebook by running the cell below. 

In [ ]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Now, copy the paths to the life expectancy dataset and census dataset to the `le_path` and `census_path` variables respectively in the cell below. Then, run the cell to save your variables.  

In [ ]:
le_path = '/content/drive/My Drive/CIS550/life_expectancy.csv' # Path to life expectancy dataset in your drive
census_path = '/content/drive/My Drive/CIS550/adult.data' # Path to census subset in your drive

Finally, run the cell below to load the census data into a `DataFrame` named `census` and the life expectancy data into a `DataFrame` named `le`. 

In [ ]:
col_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'maritial-status',
             'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
             'hours-per-week', 'native-country', 'income']
census = pd.read_csv(census_path, header=None, names=col_names)
le = pd.read_csv(le_path)
le.head()

,country,year,le_birth,le_birth_male,le_birth_female,le_60,le_60_male,le_60_female,hle_birth,hle_birth_male,hle_birth_female,hle_60,hle_60_male,hle_60_female
0,Afghanistan,2016,62.7,61.0,64.5,16.3,15.5,17.1,53.0,52.1,54.1,11.3,10.9,11.7
1,Afghanistan,2015,63.2,61.8,64.7,16.3,15.5,17.1,53.2,52.6,54.1,11.2,10.8,11.6
2,Afghanistan,2014,63.0,61.7,64.4,16.2,15.4,17.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2013,62.7,61.5,64.1,16.2,15.4,16.9,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2012,62.2,60.9,63.6,16.1,15.3,16.8,NaN,NaN,NaN,NaN,NaN,NaN


###  Life Expectancy Dataset Column Meanings
In the life expectancy dataset above, `le` stands for "life expectancy", `hle` stands for "healthy life expectancy". Life expectancy means the number of years a person can expect to live, and healthy life expectancy means the number of years a person can be in good health. 

The columns with `60` in their names give life expectancy or healthy life expectancy for 60 year olds living in the given country at the given year. The columns with `birth` in their names give life expectancy or healthy life expectancy for individuals born in the given country in the given year. 

The columns with the `male` suffix give life expectancy statistics for men. The columns with `female` suffix give life expectancy statistics for women. And the column with no `male` or `female` suffix give statistics for an arbitrary person. 

Now, let's start our pre-processing!

## Clean Missing Values

For application quality control, we would like to ensure our database doesn't contain any missing values. 

As we saw in the EDA tutorial, the census dataset has missing values in `workclass`, `occupation`, and `native-country`. Similar analysis of the life expectancy data indicates all the `hle` columns also have missing values (encoded as NaN's). 

To resolve these problems, we will: 
1. Remove all rows from the census dataset that have at least one missing value, and 
2. Replace the missing `hle` values with appropriate defaults.

Let's see how. 

### Remove Rows with Missing Values
Let's remove all rows from the census dataset that contain at least one missing value. 

*Recall from the EDA tutorial that the census dataset denotes missing values with " ?".*


First, we generate a dataframe where each element indicates whether that element was missing in the original dataframe.

In [ ]:
missing_census = census.isin([" ?"])
missing_census.head()

,age,workclass,fnlwgt,education,education-num,maritial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Next, we collapse the dataframe of missing indicators into a `Series` of indicators giving whether *any* of the values in each row are missing. 

In [ ]:
missing_census = missing_census.any(axis=1)

Finally, we use this `Series` to subset the original census dataset to only contain those rows where there aren't any missing values. 

*Note: Applying `~` to a boolean `Series` negates the truth value of each element*

In [ ]:
census = census.loc[(~missing_census).values, :]

As a sanity check, we ensure none of the elements in `census` equal " ?" now.

In [ ]:
census.isin([" ?"]).any().any()

False

Great! It looks like we dropped all the rows with missing values from the census dataset. Now, let's turn our attention to the life expectancy data. 

### Replace Missing Values with Defaults
For the life expectancy dataset, more than 70% of rows have at least 1 missing value. But only the healthy life expectancy values are missing. All of the vanilla life expectancy statistics are in tact. As a result, throwing out all of the rows with missing values would be very costly.

Instead, we'll replace all missing values with reasonable defaults based on known values. Our replacement formula takes advantage of two facts:

1. Every `hle` statistic has an `le` counterpart (e.g. `hle_60_male` and `le_60_male`). This allows us to express each `hle` value as $\text{hle}=\frac{\text{hle}}{\text{le}}\cdot \text{le}$. Let `hle_prop` denote $\frac{\text{hle}}{\text{le}}$. This value represents the portion of their expected lifetime a person can expect to remain healthy for. 

  Let's make this concrete with an example. Consider life expectancy and healthy life expectancy for 60 year-old men in Afghanistan in 2016. `hle_60_male=10.9` and `le_male_60=15.5`, so `hle_prop` $=\frac{\text{hle}}{\text{le}}=\frac{10.9}{15.5}=0.70$. In words, a 60 year-old man in Afghanistan in 2016 can expect to spend about 70% of his remaining life healthy. 

2. In every case there's a missing `hle` statistic, the corresponding `le` statistic is known. 


Using these facts, we compute default value for a missing `hle` value as the product of the known `le` value for the focal row and the mean `hle_prop` value, computed using all rows where the relevant `le` and `hle` values are known. 

This formula is unbiased under the assumption that the relationship between life expectancy and healthy life expectancy is the same for country-year pairs where healthy life expectancy is known and pairs where it's unknown. 


Now, let's actually compute the formula. 

First, we make a map of `hle` statistics to corresponding `le` statistics. 

In [ ]:
col_map = dict()
for col in le.columns:
  if "hle" in col:
    col_map[col] = col[1:]  # removing the first letter
col_map

{'hle_60': 'le_60',
 'hle_60_female': 'le_60_female',
 'hle_60_male': 'le_60_male',
 'hle_birth': 'le_birth',
 'hle_birth_female': 'le_birth_female',
 'hle_birth_male': 'le_birth_male'}

Now, we compute the mean `hle_prop` for each `hle` statistic

In [ ]:
prop_map = dict()
for hle_col, le_col in col_map.items():
  prop_map[hle_col] = (le[hle_col] / le[le_col]).mean(skipna=True)
prop_map

{'hle_60': 0.7596736425993857,
 'hle_60_female': 0.7565391913858537,
 'hle_60_male': 0.7634433440990085,
 'hle_birth': 0.8796379697074018,
 'hle_birth_female': 0.8734263702625841,
 'hle_birth_male': 0.8860401169946647}

Next, we extract the indices of the rows with missing values for each `hle` statistic. 

In [ ]:
missing_map = dict()
for hle_col in col_map.keys():
  is_missing = le[hle_col].isna()
  missing_map[hle_col] = le.index[is_missing]

Finally, we iterate over the `hle` statistics and replace each missing value with the corresponding `le` statistic multiplied by the corresponding proportion. 



In [ ]:
for hle_col, le_col in col_map.items():
  le.loc[missing_map[hle_col], hle_col] = le.loc[missing_map[hle_col], le_col] * prop_map[hle_col]
le.head()

,country,year,le_birth,le_birth_male,le_birth_female,le_60,le_60_male,le_60_female,hle_birth,hle_birth_male,hle_birth_female,hle_60,hle_60_male,hle_60_female
0,Afghanistan,2016,62.7,61.0,64.5,16.3,15.5,17.1,53.000000,52.100000,54.100000,11.300000,10.900000,11.700000
1,Afghanistan,2015,63.2,61.8,64.7,16.3,15.5,17.1,53.200000,52.600000,54.100000,11.200000,10.800000,11.600000
2,Afghanistan,2014,63.0,61.7,64.4,16.2,15.4,17.0,55.417192,54.668675,56.248658,12.306713,11.757027,12.861166
3,Afghanistan,2013,62.7,61.5,64.1,16.2,15.4,16.9,55.153301,54.491467,55.986630,12.306713,11.757027,12.785512
4,Afghanistan,2012,62.2,60.9,63.6,16.1,15.3,16.8,54.713482,53.959843,55.549917,12.230746,11.680683,12.709858


As a sanity check, let's make sure there are no `NaN`'s left in the dataframe

In [ ]:
le.isna().any().any()

False

### Remove rows, replace with defaults, or ignore?
None of these options is strictly better than the others. Depending on your application and what exactly is missing from the data, any choice could be acceptable.

This doesn't mean you can make the decision thoughtlessly. In context, certain choices can be misguided. For example, you can't ignore missing values in a column you plan to use as a table index, and you can't throw out all rows with missing values if 95% of your rows are incomplete. 

So for the project, think carefully about which approach is appropriate for your application and data. Feel free to have a discussion with your project mentor. We expect you to justify whatever decision you make in your project report. 

## Entity Resolution

Now that we've handled missing values in both datasets, we turn our attention to performing entity resolution on the entities common to both. In this case, those common entities are countries.

To perform entity resolution, we will:
1. Determine whether both datasets use the same names to refer to all countries
2. Edit the names in one dataset to match the other, if necessary

In the general case, you may also need to detect when datasets refer to different entities using the same name/ID and disambiguate these references. This may happen when handling datasets that contain multiple people with the same name, for example. But we don't need to worry about it here because the names of countries are well-known and distict. 


### Detect Inconsistent Names
Let's compile a list of all country names in both datasets, then inspect it for repetitions. 

First, we extract the unique names of countries from both datasets.

In [ ]:
census_countries = census["native-country"].unique()
le_countries = le["country"].unique()


Now, we combine these into one set.

In [ ]:
countries = set(census_countries.tolist() + le_countries.tolist())

Let's inspect the contents of the set. The output is automatically sorted.

In [ ]:
countries

{' Cambodia',
 ' Canada',
 ' China',
 ' Columbia',
 ' Cuba',
 ' Dominican-Republic',
 ' Ecuador',
 ' El-Salvador',
 ' England',
 ' France',
 ' Germany',
 ' Greece',
 ' Guatemala',
 ' Haiti',
 ' Holand-Netherlands',
 ' Honduras',
 ' Hong',
 ' Hungary',
 ' India',
 ' Iran',
 ' Ireland',
 ' Italy',
 ' Jamaica',
 ' Japan',
 ' Laos',
 ' Mexico',
 ' Nicaragua',
 ' Outlying-US(Guam-USVI-etc)',
 ' Peru',
 ' Philippines',
 ' Poland',
 ' Portugal',
 ' Puerto-Rico',
 ' Scotland',
 ' South',
 ' Taiwan',
 ' Thailand',
 ' Trinadad&Tobago',
 ' United-States',
 ' Vietnam',
 ' Yugoslavia',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'C

Immediately, we see that many of the names have spaces prepended to them that appear to be preventing matches. Let's print the lists of names from each dataset separately to see where these are coming from. 

In [ ]:
print("Census Countries")
print(census_countries)
print("")
print("LE Countries")
print(le_countries)

Census Countries
[' United-States' ' Cuba' ' Jamaica' ' India' ' Mexico' ' Puerto-Rico'
 ' Honduras' ' England' ' Canada' ' Germany' ' Iran' ' Philippines'
 ' Poland' ' Columbia' ' Cambodia' ' Thailand' ' Ecuador' ' Laos'
 ' Taiwan' ' Haiti' ' Portugal' ' Dominican-Republic' ' El-Salvador'
 ' France' ' Guatemala' ' Italy' ' China' ' South' ' Japan' ' Yugoslavia'
 ' Peru' ' Outlying-US(Guam-USVI-etc)' ' Scotland' ' Trinadad&Tobago'
 ' Greece' ' Nicaragua' ' Vietnam' ' Hong' ' Ireland' ' Hungary'
 ' Holand-Netherlands']

LE Countries
['Afghanistan' 'Albania' 'Algeria' 'Angola' 'Antigua and Barbuda'
 'Argentina' 'Armenia' 'Australia' 'Austria' 'Azerbaijan' 'Bahamas'
 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus' 'Belgium' 'Belize' 'Benin'
 'Bhutan' 'Bolivia (Plurinational State of)' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Brunei Darussalam' 'Bulgaria' 'Burkina Faso'
 'Burundi' 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada'
 'Central African Republic' 'Chad' 'Chile' 'China' 'Colombia' 'C

The output above indicates the names with spaces are coming from the census dataset. So let's remove those spaces, recompile the list, and take look for countries that appear twice in the output.

In [ ]:
fixed_census_countries = list()
for country in census_countries.tolist():
  if country[0] == " ":
    fixed_census_countries.append(country[1:])
countries = set(fixed_census_countries + le_countries.tolist())
countries

{'Afghanistan',
 'Albania',
 'Algeria',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Columbia',
 'Comoros',
 'Congo',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 "Democratic People's Republic of Korea",
 'Democratic Republic of the Congo',
 'Denmark',
 'Djibouti',
 'Dominican Republic',
 'Dominican-Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'El-Salvador',
 'England',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany'

The output contains 9 countries with multiple names:

*   "United States of America" and "United-States"
*   "Trinidad and Tobago" and "Trinidad&Tobago"
*   "Dominican Republic" and "Dominican-Republic"
*   "El-Salvador" and "El Salvador"
*   "Columbia" and "Colombia"
*   "Netherlands" and "Holand-Netherlands"
*   "Laos" and "Lao People's Democratic Republic"
*   "Iran" and "Iran (Islamic Republic of)"
*   "Viet Nam" and "Vietnam"

Relatedly, the output contains an entry for the United Kingdom, as well as entries for Scotland and England, which are part of the UK. 

*Republic of Korea and the Democratic People's Republic of Korea are different countries. Congo and Democratic Republic of the Congo are also different.* 

### Resolve Inconsistent Names
Now, let's fix the problems the we uncovered in the previous step.

First, we trim the whitespace off the start and end of the country names in both datasets to handle that space issue.

In [ ]:
census["native-country"] = census["native-country"].str.strip()
le["country"] = le["country"].str.strip()

For each country with multiple names, we need to purge one of the names from the dataset. We'll do this by replacing all usages of one name with the other. 

We'll also replace all substitute all usages Scotland and England for the U.K. This solution sacrifices some precision, but at least we avoid the incorrect conclusion that there's no valid counterpart for rows containing Scotland and England.  

To accomplish this, we first create a `Series` of the countries with multiple names, where of each element is the name we'll purge, and the value is name we'll keep.

In [ ]:
name_map = {
     "United-States": "United States of America",
     "Trinadad&Tobago": "Trinidad and Tobago",
     "Dominican-Republic": "Dominican Republic",
     "Columbia" : "Colombia",
     "Lao People's Democratic Republic": "Laos",
     "Scotland": "United Kingdom of Great Britain and Northern Ireland",
     "England": "United Kingdom of Great Britain and Northern Ireland",
     "Iran (Islamic Republic of)": "Iran",
     "Holand-Netherlands": "Netherlands",
     "Viet Nam" : "Vietnam",
     "El-Salvador": "El Salvador"
}
name_series = pd.Series(data=list(name_map.values()), index=list(name_map.keys()))
name_series

United-States                                                United States of America
Trinadad&Tobago                                                   Trinidad and Tobago
Dominican-Republic                                                 Dominican Republic
Columbia                                                                     Colombia
Lao People's Democratic Republic                                                 Laos
Scotland                            United Kingdom of Great Britain and Northern I...
England                             United Kingdom of Great Britain and Northern I...
Iran (Islamic Republic of)                                                       Iran
Holand-Netherlands                                                        Netherlands
Viet Nam                                                                      Vietnam
El-Salvador                                                               El Salvador
dtype: object

Next, we find the indices of all usages of the names we're purging in both datasets

In [ ]:
purge_list = list(name_map.keys())
census_idx = census.index[census["native-country"].isin(purge_list)]
le_idx = le.index[le["country"].isin(purge_list)]

Using these indices, we extract the names that we need to update as `Series`

In [ ]:
census_problems = census.loc[census_idx, "native-country"]
le_problems = le.loc[le_idx, "country"]

Next, we replace the problematic names in these `Series` with the appropriate counterparts by indexing `name_series` with them. 

*The operations in the cell below replace the names correctly because `name_series` maps each name we wanted to purge to the we wanted to replace it with.*

In [ ]:
census_fixed = name_series.loc[census_problems.values].values
le_fixed = name_series.loc[le_problems.values].values 

Finally, we update the dataframes with the fixed names.

In [ ]:
census.loc[census_idx, "native-country"] = census_fixed
le.loc[le_idx, "country"] = le_fixed

### Remove Unpaired Entities
In some cases, you may want to exclude entities that only appear in one dataset or the other from your database. Let's suppose that's the case here and remove all countries that only appear in one dataset or the other. 

As before, we first extract the full list of countries found in both datasets.

In [ ]:
census_countries = census["native-country"].unique().tolist()
le_countries = le["country"].unique().tolist()

Then, we convert these lists to sets and use set-difference operations to find the countries that only appear in one set. 

In [ ]:
census_countries = set(census_countries)
le_countries = set(le_countries)
cen_diff = census_countries.difference(le_countries)
le_diff = le_countries.difference(census_countries)
total_diff = list(cen_diff) + list(le_diff)

Finally, we drop all rows from both datasets that contain countries that match any of the countries in this list. 

In [ ]:
le = le.loc[~le["country"].isin(total_diff), :]
census = census.loc[~census["native-country"].isin(total_diff), :]

## Replace Categorical Variables with Indicators

Depending on your application, you may want to replace some of your text-based categorical variables with numeric equivalents. This substitution can reduce the runtime of some queries and decrease the your tables require. It can be especially fruitful for columns you plan to use as indices.  

Let's see an example. Below, we'll convert the columns containing countries in our datasets to numeric equivalents. 

First, we create a `Series` that maps each country name to an integer. 


In [ ]:
all_countries = le["country"].unique()
country_codes = pd.Series(index=all_countries, data=np.arange(len(all_countries)))
country_codes

Cambodia                                                 0
Canada                                                   1
China                                                    2
Colombia                                                 3
Cuba                                                     4
Dominican Republic                                       5
Ecuador                                                  6
El Salvador                                              7
France                                                   8
Germany                                                  9
Greece                                                  10
Guatemala                                               11
Haiti                                                   12
Honduras                                                13
Hungary                                                 14
India                                                   15
Iran                                                    

Next, we use this `Series` to map all countries in both datasets to the corresponding integers by indexing the `Series` with the names of the countries in the datasets. 

In [ ]:
le["country"] = country_codes.loc[le["country"]].values
census["native-country"] = country_codes.loc[census["native-country"]].values

,age,workclass,fnlwgt,education,education-num,maritial-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,32,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,32,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,32,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,32,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,4,<=50K


Finally, we convert the columns to integer types. 

In [ ]:
le["country"] = le["country"].astype(np.int64)
census["native-country"] = census["native-country"].astype(np.int64)


## Find an Index

Before ingesting our data into the database, we need to find a unique index for each table. Let's do this for the life expectancy dataset. 

### Single Column Index
The fastest way to determine whether any single column is unique is to check whether the number of unique values in the candidate column equals the number of elements. For example, let's find out whether `country` could be the index.  

In [ ]:
len(le["country"].unique()) == len(le["country"])

False

### Multi-Column Index

When there's not an individual column that can act as an index, we search for combinations of columns that can make a unique index when combined. 

To check a candidate set of columns:
1. Call `DataFrame.groupby()` on the list of candidate columns. *This creates a group of rows for each unique combination of candidate olumn values that appears in the Dataframe*
2. Call `GroupBy.size()` on the resulting grouped dataframe. *This counts the number of rows in each group*
3. Check whether every group has exactly 1 row. 

We use this procedure below to check whether `country` and `year` can function as a joint index. 

In [ ]:
# Step 1: Group dataframe by candidate columns 
grouped_le = le.groupby(["country", "year"]) 
# Step 2: Count rows in each group
counts = grouped_le.size()
print(counts)
# Step 3: Check whether every value equals 1
(counts == 1).all()

country  year
0        2000    1
         2001    1
         2002    1
         2003    1
         2004    1
                ..
33       2012    1
         2013    1
         2014    1
         2015    1
         2016    1
Length: 578, dtype: int64


True

Great! This means no country-year pair appears more than once in the life expectancy table, so we can use `country` and `year` in combination as the index. 

For the census dataset, you'd need every column to create a unique index (Check this for yourself). So we'll just plan to use the arbitrary, unique integers `census.index` as our table index. 

## Export Data
After cleaning our datasets, resolving entity resolution problems, and choosing indices, we're ready to ingest our the data into our database.

In the past, students have found using Python for data ingestion slow and frustrating, so we won't populate the database here. Instead, we'll export both datasets and the country codes to CSVs. Then, we'll show you how to ingest these CSVs into your database using MySQL Workbench in the tutorial on data ingestion. 


First, convert the `Series` of country codes to an equivalent `DataFrame`

In [ ]:
country_codes = pd.DataFrame(data={'country': country_codes.index, 'code': country_codes.values})

Next, we use `DataFrame.to_csv` to write each dataset to a CSV file with a descriptive name. 

*For `le` and `country_codes`, we set `index` to `False` because the indices of the `DataFrames` are meaningless integers that we don't need in our tables. For `census`, we rename the index and include it in the output because we decided  to use it as our table index, even though it's arbitrary.*

In [ ]:
le.to_csv("le.csv", index=False)
country_codes.to_csv("country_codes.csv", index=False)
census.index.name = "id"
census.to_csv("census.csv", index=True)

NameError: ignored

Finally, we download these files to our local machine, so we can put them into MySQL Workbench later. 

In [ ]:
from google.colab import files
files.download('census.csv') 
files.download("country_codes.csv")
files.download("le.csv")

## Exercises
Check out [these exercises](https://drive.google.com/open?id=1kjLaYC_KJUlltm-iAzgF5VmHgb7Uer0z) to practice the processing techniques you learned above!